In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset
from lib.ehr.icu_interface import Inpatients

In [3]:
import logging
logging.root.level = logging.DEBUG


In [4]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

with U.modified_environ(DATA_DIR=DATA_DIR):
    m4inpatient_dataset = load_dataset('M4ICU')
   

DEBUG:root:Loading dataframes
/home/asem/GP/ICE-NODE/notebooks/mimic_icu/../../lib/ehr/dataset.py:494: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["adm"][colname["adm"][time_col]] = pd.to_datetime(
/home/asem/GP/ICE-NODE/notebooks/mimic_icu/../../lib/ehr/dataset.py:494: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["adm"][colname["adm"][time_col]] = pd.to_datetime(
/home/asem/GP/ICE-NODE/notebooks/mimic_icu/../../lib/ehr/dataset.py:501: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict

INFO:root:Unrecognised ICD v9 codes: 118 (1.63%)
DEBUG:root:
                    Unrecognised <class 'lib.ehr.coding_scheme.DxICD9'> codes (118)
                    to be removed: ['041.49', '173.21', '173.22', '173.30', '173.31', '173.32', '173.40', '173.41', '173.42', '173.50', '173.51', '173.52', '173.59', '173.60', '173.61', '173.62', '173.70', '173.71', '173.72', '173.79', '173.80', '173.81', '173.82', '173.91', '173.92', '173.99', '282.40', '282.43', '282.44', '282.46', '284.11', '284.12', '284.19', '286.52', '286.53', '286.59', '294.20', '294.21', '310.81', '310.89', '331.6', '348.82', '358.30', '365.70', '365.72', '365.73', '414.4', '415.13', '425.11', '425.18', '444.09', '488.81', '488.82', '488.89', '512.2', '512.82', '512.83', '512.84', '512.89', '516.31', '516.32', '516.33', '516.34', '516.35', '516.36', '516.37', '516.4', '516.5', '518.51', '518.52', '518.53', '539.01', '539.09', '539.81', '539.89', '573.5', '596.81', '596.82', '596.83', '596.89', '629.31', '649.81', '704.

In [5]:
splits = m4inpatient_dataset.random_splits(0.8, 0.9, 42)

In [9]:
import jax
jax.config.update('jax_platform_name', 'cpu')
m4inaptients = Inpatients(m4inpatient_dataset, splits[2][:32])

Observables:  44%|███████████████████████████████████████████████████████████████████▌                                                                                     | 38/86 [00:12<00:04, 10.19it/s]WARNING:jax._src.dispatch:Finished tracing + transforming jit(broadcast_in_dim) in 0.00028705596923828125 sec
DEBUG:jax._src.interpreters.pxla:Compiling broadcast_in_dim for with global shapes and types [ShapedArray(float32[])]. Argument mapping: (GSPMDSharding({replicated}),).
Observables: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:19<00:00,  4.34it/s]
DEBUG:jax._src.interpreters.pxla:Compiling convert_element_type for with global shapes and types [ShapedArray(bool[5])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.interpreters.pxla:Compiling concatenate for with global shapes and types [ShapedArray(float32[5], weak_type=True), ShapedArray(f

In [11]:
m4inaptients.subjects

[Inpatient(
   subject_id='10514954',
   static_info=StaticInfo(
     gender='F',
     date_of_birth=Timestamp('2079-01-01 00:00:00'),
     ethnicity=bool[5],
     ethnicity_scheme=<lib.ehr.coding_scheme.MIMIC4Eth5 object at 0x7fdf2816a770>,
     constant_vec=f32[6]
   ),
   admissions=[
     InpatientAdmission(
       admission_id='23468474',
       admission_dates=(
         Timestamp('2168-12-15 23:42:00'),
         Timestamp('2168-12-22 13:45:00')
       ),
       dx_codes=Codes(
         codes=set(),
         vec=f32[17375],
         scheme=<lib.ehr.coding_scheme.DxICD9 object at 0x7fdcbb5bbd60>
       ),
       dx_codes_history=Codes(
         codes=set(),
         vec=f32[17375],
         scheme=<lib.ehr.coding_scheme.DxICD9 object at 0x7fdcbb5bbd60>
       ),
       outcome=Codes(
         codes=set(),
         vec=bool[2081],
         scheme=<lib.ehr.outcome.OutcomeExtractor object at 0x7fdf282b4e20>
       ),
       procedures=InpatientSegmentedInput(
         time_segments=[

In [15]:
from lib.ehr.coding_scheme import DxICD9, DxICD10

In [16]:
s = DxICD10()

In [17]:
len(s)

45869

## TODO

1. Squeeze code vectors.
2. Squeeze float32 to float16.

In [18]:
!ls  /home/asem/GP/ICE-NODE/lib/ehr/resources/

2018_gem_cm_I10I9.txt.gz   icd10cm_tabular_2023.xml.gz
2018_gem_cm_I9I10.txt.gz   icd10pcs_codes_2023.txt.gz
2018_gem_pcs_I10I9.txt.gz  mimic4_int_grouper_input.csv.gz
2018_gem_pcs_I9I10.txt.gz  mimic4_int_grouper_proc.csv.gz
CCS			   mimic4_obs_codes.csv.gz
CPRD_212_LTC_ALL.csv.gz    mimic4_race_grouper.csv.gz
cprd_eth.csv		   outcome_filters
HOM-ICD9.csv.gz


In [19]:
df = pd.read_csv('/home/asem/GP/ICE-NODE/lib/ehr/resources/mimic4_int_grouper_proc.csv.gz')

In [21]:
df.group.nunique()

52